In [1]:
import multiprocessing
from ..evals.multiprocessing_helper import run_agent_execution

ImportError: attempted relative import with no known parent package

In [ ]:
# A list of independent queries to be executed in parallel.
queries = [
    "How many NBA players have scored 60 or more points in a regular season game since 2023?",
    "What is the current age of the last American experimental physicist to win the Nobel Prize in Physics?",
    "How many books has Jamie Oliver published since 2020?",
]

processes = []
results = []
queue = multiprocessing.Queue()

print("🏁 Starting agent executions, each in its own isolated process...")
for i, query in enumerate(queries):
    # Create a Process for each query.
    process = multiprocessing.Process(
        target=run_agent_execution,
        args=(query, queue),  # Pass the queue to the function
        name=f"AgentProcess-{i+1}"
    )
    processes.append(process)
    process.start()  # Start the new process

# Wait for all processes to complete their execution
for process in processes:
    process.join()

# Retrieve results from the queue
while not queue.empty():
    result = queue.get()
    results.append(result)

print("Results:", results)